# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = False

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 480, 'label': 'phishing'},
        {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 3072
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/validation.parquet')

## Run classification

In [3]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/validation.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/validation.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Function to map labels to 'benign' or 'malign'
def map_label(label):
    if label == 'benign':
        return 'benign'
    else:
        return f'malign ({label})'

# Function to convert labels to binary classes
def binary_label(label):
    return 'negative' if label == 'benign' else 'positive'

# Apply label mapping
input_df['mapped_label'] = input_df['label'].apply(map_label)
input_df['binary_label'] = input_df['label'].apply(binary_label)

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Initialize counters for overall statistics
total_true_labels = []
total_pred_labels = []

# Format string for aligned output
header_format_str = "{:<5} | {:<50} | {:<20} | {:<10} | {}"
data_format_str = "{:<5} | {:<50} | {:<20} | {:<10} | {:.2f}"

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label', 'mapped_label', 'binary_label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    # Collect predictions and true labels
    true_labels = chunk_df['binary_label'].values
    pred_labels = []
    for result in chunk_results:
        pred_label = 'negative' if result['aggregate_probability'] < 0.5 else 'positive'
        pred_labels.append(pred_label)

    # Update overall statistics
    total_true_labels.extend(true_labels)
    total_pred_labels.extend(pred_labels)

    # Display header for results
    print(header_format_str.format("Status", "Domain Name", "Actual Label", "Predicted", "Probability"))

    # Display results for each domain
    for idx, result in enumerate(chunk_results):
        actual_label = chunk_df.iloc[idx]['mapped_label']
        predicted_label = 'benign' if pred_labels[idx] == 'negative' else 'malign'
        domain_name = result['domain_name']
        aggregate_probability = result['aggregate_probability']
        status = "OK" if pred_labels[idx] == true_labels[idx] else "ER"
        print(data_format_str.format(status, domain_name[:50], actual_label, predicted_label, aggregate_probability))

    # Calculate metrics for the current chunk
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, pos_label='positive', average='binary')
    recall = recall_score(true_labels, pred_labels, pos_label='positive', average='binary')
    f1 = f1_score(true_labels, pred_labels, pos_label='positive', average='binary')

    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=['negative', 'positive']).ravel()
    false_positives = fp
    false_negatives = fn
    total_positives = tp + fp
    total_negatives = tn + fn

    fp_ratio = (false_positives / total_positives) if total_positives > 0 else 0
    fn_ratio = (false_negatives / total_negatives) if total_negatives > 0 else 0

    print(f"Chunk {i+1}/{num_chunks} metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"False Positives: {false_positives} ({fp_ratio * 100:.2f}%)")
    print(f"False Negatives: {false_negatives} ({fn_ratio * 100:.2f}%)")
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

# Calculate overall metrics
overall_accuracy = accuracy_score(total_true_labels, total_pred_labels)
overall_precision = precision_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_recall = recall_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_f1 = f1_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')

overall_tn, overall_fp, overall_fn, overall_tp = confusion_matrix(total_true_labels, total_pred_labels, labels=['negative', 'positive']).ravel()
overall_false_positives = overall_fp
overall_false_negatives = overall_fn
overall_total_positives = overall_tp + overall_fp
overall_total_negatives = overall_tn + overall_fn

overall_fp_ratio = (overall_false_positives / overall_total_positives) if overall_total_positives > 0 else 0
overall_fn_ratio = (overall_false_negatives / overall_total_negatives) if overall_total_negatives > 0 else 0

print("Overall metrics:")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")
print(f"Overall False Positives: {overall_false_positives} ({overall_fp_ratio * 100:.2f}%)")
print(f"Overall False Negatives: {overall_false_negatives} ({overall_fn_ratio * 100:.2f}%)")


2024-06-05 15:28:24.365129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 15:28:24.365155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 15:28:24.365906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 15:28:24.370051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 15:28:25.031252: W tensorflow/compiler/tf2

===== Processing chunk 1/103 =====
Status | Domain Name                                        | Actual Label         | Predicted  | Probability
ER    | aspersasnails.com                                  | benign               | malign     | 0.95
ER    | www.ipop.at                                        | benign               | malign     | 0.94
OK    | qwxjlfffjtlllyllyg.ms                              | malign (dga)         | malign     | 0.85
ER    | truongdaylaixe.vn                                  | benign               | malign     | 0.95
ER    | gw-api.schneider-electric.com                      | benign               | malign     | 0.95
OK    | fjz.fm                                             | malign (dga)         | malign     | 0.93
OK    | eduroam.mendelu.cz                                 | benign               | benign     | 0.00
OK    | mbubebxogjudywh.mn                                 | malign (dga)         | malign     | 0.95
ER    | readydesigns.com               

## Optional: Generate preliminary results for training the final aggregation classifier

In [5]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=True)

In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod,badness_probability,phishing_xgboost_result,phishing_deepnn_result
0,pagesclicks.pages.dev,0.900000,0.575000,1.000000,0.791667,1.000000,0.875000,1.000000,0.833333,1.000000,1.000000,phishing,1.000000,0.999994,0.000000,0.000145,0.000000,1.999994,0.999997,0.999994,0.000145,0.000072,0.000000,2.000140,0.400028,0.000000,0.999991,0.999984,1.000000
1,tt-12340-4.seg.t.tailtarget.com,0.600000,0.200000,1.000000,0.833333,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,benign,0.000000,0.993445,0.000000,0.097857,0.000000,0.993445,0.496722,0.000000,0.097857,0.048929,0.000000,1.091302,0.218260,0.000000,0.999985,0.987218,0.701366
2,widgets.infor.pl,0.600000,0.200000,1.000000,0.833333,1.000000,0.500000,1.000000,0.708333,1.000000,0.944444,benign,0.000000,0.163195,0.000000,0.007403,0.000000,0.163195,0.081598,0.000000,0.007403,0.003702,0.000000,0.170599,0.034120,0.000000,0.001833,0.334656,0.007648
3,wealthharbor.pro,0.900000,0.550000,1.000000,0.791667,1.000000,0.750000,1.000000,0.750000,1.000000,1.000000,malware,1.000000,0.999928,1.000000,1.000000,0.012438,1.999928,0.999964,0.999928,2.000000,1.000000,1.000000,4.012366,0.802473,0.012438,1.000000,0.999699,1.000000
4,lefakap.cz,0.950000,0.600000,1.000000,0.750000,1.000000,0.625000,1.000000,0.458333,1.000000,0.722222,benign,0.000000,0.999531,0.000000,0.999895,0.128405,0.999531,0.499766,0.000000,0.999895,0.499948,0.000000,2.127831,0.425566,0.000000,1.000000,0.995793,0.199939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,www.costar.com,0.600000,0.225000,1.000000,0.791667,1.000000,0.375000,1.000000,0.708333,1.000000,0.666667,benign,0.000000,0.174444,0.000000,0.033922,0.000000,0.174444,0.087222,0.000000,0.033922,0.016961,0.000000,0.208366,0.041673,0.000000,0.021362,0.183661,0.048096
3068,www.hourova.cz,0.600000,0.200000,1.000000,0.750000,1.000000,0.625000,1.000000,0.625000,1.000000,0.666667,benign,1.000000,0.999855,0.000000,0.999605,0.000000,1.999855,0.999928,0.999855,0.999605,0.499803,0.000000,2.999460,0.599892,0.000000,1.000000,0.997345,0.893651
3069,rhdxgdkvaoxnyqqdubk.tj,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,1.000000,0.025671,1.000000,0.510772,0.999983,1.025671,0.512836,0.025671,1.510772,0.755386,0.510772,3.536425,0.707285,0.013112,0.999995,0.004352,0.027265
3070,interskin.cyou,0.900000,0.550000,1.000000,0.833333,1.000000,0.750000,1.000000,0.750000,1.000000,1.000000,malware,1.000000,0.999980,1.000000,1.000000,0.000010,1.999980,0.999990,0.999980,2.000000,1.000000,1.000000,3.999990,0.799998,0.000010,1.000000,0.999992,0.999982


In [7]:
preliminary_results_df[["domain_name", "label", "badness_probability", "phishing_cnn_result", "phishing_deepnn_result", "phishing_lgbm_result", "phishing_xgboost_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,badness_probability,phishing_cnn_result,phishing_deepnn_result,phishing_lgbm_result,phishing_xgboost_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,pagesclicks.pages.dev,phishing,0.999991,1.000000,1.000000,0.999994,0.999984,0.000000,0.000145,0.000000
1,tt-12340-4.seg.t.tailtarget.com,benign,0.999985,0.000000,0.701366,0.993445,0.987218,0.000000,0.097857,0.000000
2,widgets.infor.pl,benign,0.001833,0.000000,0.007648,0.163195,0.334656,0.000000,0.007403,0.000000
3,wealthharbor.pro,malware,1.000000,1.000000,1.000000,0.999928,0.999699,1.000000,1.000000,0.012438
4,lefakap.cz,benign,1.000000,0.000000,0.199939,0.999531,0.995793,0.000000,0.999895,0.128405
...,...,...,...,...,...,...,...,...,...,...
3067,www.costar.com,benign,0.021362,0.000000,0.048096,0.174444,0.183661,0.000000,0.033922,0.000000
3068,www.hourova.cz,benign,1.000000,1.000000,0.893651,0.999855,0.997345,0.000000,0.999605,0.000000
3069,rhdxgdkvaoxnyqqdubk.tj,dga,0.999995,1.000000,0.027265,0.025671,0.004352,1.000000,0.510772,0.999983
3070,interskin.cyou,malware,1.000000,1.000000,0.999982,0.999980,0.999992,1.000000,1.000000,0.000010


In [20]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

,domain_name,label,badness_probability
0,est1.hlsvideocdn78.shop,benign,1.000000
1,hp18.pages.dev,phishing,0.999990
2,www.charlotteobserver.com,benign,0.000260
3,zakazky.pardubickykraj.cz,benign,0.000168
4,vid-s16-n10-nl-cdn.eporner.com,benign,0.000192
5,zgktpslfgyflqi.com,dga,1.000000
6,www.abanca.com,benign,0.000777
7,reviewholdreviewho.wixsite.com,phishing,0.999996
8,covers.alibrate.com,benign,0.999995
9,lfjvcjow.org,dga,0.999999
